In [ ]:
%pip install transformers datasets evaluate

In [ ]:
from huggingface_hub import notebook_login

In [ ]:
notebook_login()

In [ ]:
from datasets import load_dataset

In [ ]:
sandra = load_dataset('_training-data')

In [ ]:
data = sandra['train'].train_test_split(test_size=0.15)

In [ ]:
data['train'][0]

In [ ]:
def preprocess_function(examples):
    from transformers import AutoTokenizer
    tokenizer = AutoTokenizer.from_pretrained('decapoda-research/llama-7b-hf')
    return tokenizer([" ".join(x) for x in examples['message']])

In [ ]:
tokenized = data.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=data['train'].column_names
)

In [ ]:
# def preprocess_function(examples):
#     from transformers import AutoTokenizer
#     tokenizer = AutoTokenizer.from_pretrained('decapoda-research/llama-7b-hf')

#     tokenizer.pad_token = tokenizer.eos_token

#     return tokenizer.batch_encode_plus(
#         [x for x in examples['message']],
#         padding=True,
#         truncation=True,
#         max_length=512,
#         return_tensors='pt',
#         pad_to_max_length=True
#     )

# tokenized = data.map(
#     preprocess_function,
#     batched=True,
#     num_proc=4,
#     remove_columns=data["train"].column_names,
# )

In [ ]:
def group_texts(examples):

    block_size = 128

    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])

    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result['labels'] = result['input_ids'].copy()
    return result

lm_dataset = tokenized.map(group_texts, batched=True, num_proc=4)

In [ ]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer

model = AutoModelForCausalLM.from_pretrained('decapoda-research/llama-7b-hf')

In [ ]:
training_args = TrainingArguments(
    output_dir='be-right-back',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset['train'],
    eval_dataset=lm_dataset['test'],
    data_collator=data_collator,
)

trainer.train()

In [ ]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

In [ ]:
trainer.push_to_hub()